In [16]:
import layoutparser as lp
from sklearn.metrics import precision_score
import cv2 #opencv-python-headless             4.5.4.58/ ahora opencv-python                      4.5.5.62
import pytesseract
#import easyocr
import numpy as np
import matplotlib.pyplot as plt
import time
import keyboard
from pytesseract import Output

In [11]:
pytesseract.pytesseract.tesseract_cmd = r'D:\Program Files\Tesseract-OCR\tesseract.exe'
custom_config = r'--oem 3 --psm 1 -l spa'

In [14]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(img):
    return cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)

#dilation
def dilate(image):
    kernel = np.ones((25,25),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((25,25),np.uint8)
    return cv2.erode(image, kernel, iterations = 2)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((25,25),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

#template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)

def leertxt(archivotxt):
    content = open("004.txt", "r", encoding='utf-8').read()
    lecturatxt = re.split('\s|\n',content)
    try:
        while True:
            lecturatxt.remove('')
    except ValueError:
        pass
    return lecturatxt

def listafiltrada(archivoimagen):
    img = cv2.imread(archivoimagen)
    text = pytesseract.image_to_data(img, config=custom_config, output_type=Output.DATAFRAME).dropna()
    #text['text'] = text['text'].lower()
    list = text["text"].tolist()
    try:
        while True:
            list.remove('')
    except ValueError:
        pass
    try:
        while True:
            list.remove(' ')
    except ValueError:
        pass
    list.pop(-1)
    filterlist = []
    skip = []
    for i in range(len(list)):
        if i in skip:
            continue
        if list[i].find("-")!=-1:
            if i+1>=len(list): #por si es la última palabra de la lista
                break
            else:
                filterlist.append(list[i].replace('-', '')+list[i+1])
                skip.append(i+1)
        else:
            filterlist.append(list[i])
    return filterlist

In [4]:
start = time.perf_counter()
img = cv2.imread("PruebaOrganica.png", 0)
eroded = erode(img)
dilated = dilate(eroded)
thresholded = cv2.threshold(dilated, 150, 255, cv2.THRESH_BINARY_INV)[1]
countours = cv2.findContours(thresholded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]
if len(countours) == 2:
    x, y, w, h = cv2.boundingRect(countours[0])
    crop = img[y:h + y, x:w + x]
    text = pytesseract.image_to_string(crop)
end = time.perf_counter()
print(f'Finished in {round(end-start, 2)} second(s)')

Finished in 1.24 second(s)


In [5]:
print(text)

Product : Perpetual 25-3E Calendar
Manufacturer : Op der Millen -27, avenue Guillaume Moreau - 1050 Bruxelles (Belgium)
Designer : Jean Keup

Description: This cilendat- designed and created in an old mill of Luxembourg Province -
is exported throughout the world. This collection - called after its creators : Op der Millen
is one example of highly elaborate Belgian design
Material: Pastified cardboard printed in 5 colors and in five languages per calenda
Dimensions : Diameter 40 em x 3
Packing: Individual cardboard case
Price per item (Including gift wrapping, with the mention “presented by.
‘export duties from Belgium).
Minimum order : $00 units.

For 500: 83 Bel. Fr.

For 1000 and more : 78 Bel. Fr.
Delivery delays: 6 to 8 weeks.

 

 

and

Design Conte. Tel 11 9341. ~ 1000- Bruxelles
Galerie Ravenstein Galeri) $1 ~ 1000 - Brussel



In [85]:
i = 0
while True:
    i += 1
    print(i)
    time.sleep(1)
    if keyboard.is_pressed("q"):
        print("q pressed, ending loop")
        break

Sleeping for 1 second(s)
Done sleeping
You pressed enter try again!
Sleeping for 1 second(s)
Done sleeping
You pressed enter try again!
Sleeping for 1 second(s)
Done sleeping
You pressed p


In [36]:
img = cv2.imread("PruebaOrganica.png", 0) #GrayScale Directamente con el 0
#img = thresholding(img)
details = pytesseract.image_to_data(img, output_type=Output.DICT, config=custom_config)
total_boxes = len(details['text'])
for sequence_number in range(total_boxes):
    if int(float(details['conf'][sequence_number])) >30:
        (x, y, w, h) = (details['left'][sequence_number], details['top'][sequence_number], details['width'][sequence_number],  details['height'][sequence_number])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

['-1', '-1', '-1', '-1', '95.000000', '-1', '-1', '-1', '91.862839', '91.862839', '93.304703', '67.582245', '96.060577', '-1', '78.470871', '89.527863', '96.873306', '96.284996', '92.663857', '80.491806', '96.629425', '96.033119', '92.834129', '81.598137', '96.557480', '96.204376', '87.936600', '-1', '94.393997', '63.474052', '63.474052', '91.896103', '-1', '-1', '-1', '58.106236', '93.026604', '39.369675', '73.345490', '89.393974', '89.393974', '96.172005', '96.172005', '96.035423', '96.035423', '95.432465', '94.322678', '93.235649', '78.921066', '-1', '94.231918', '96.371834', '96.667221', '96.983261', '83.857056', '96.781647', '72.711899', '80.825546', '80.689140', '60.608105', '60.608105', '62.495152', '91.890694', '96.862518', '96.584747', '95.429565', '-1', '61.036530', '96.615250', '96.503815', '95.005211', '93.825493', '96.387802', '95.997482', '84.558289', '-1', '64.262985', '35.535019', '90.794708', '96.953262', '93.288925', '43.193939', '96.688728', '83.896217', '83.896217',

In [39]:
img = cv2.imread("PruebaOrganica.png", 0)
text = pytesseract.image_to_string(img)
print(text)

 

Product : Perpetual 25-3E Calendar
Manufacturer : Op der Millen -27, avenue Guillaume Moreau - 1050 Bruxelles (Belgium)
Designer : Jean Keup

Description: This cdlendar- designed and created in an old mill of Luxembourg Province -
is exported throughout the world. This collection - called after its creators : Op der Millen
is one example of highly elaborate Belgian design
Material: Plastified cardboard printed in S colors and in five languages per calendar.
Dimensions: Diameter 40 em x 3
Packing: Individual cardboard case
Price per item (Including gift wrapping, with the mention “presented by.
‘export duties from Belgium).
Minimum order : $00 units.
‘or 500 : 83 Bel. Fr

For 1000 and more : 78 Bel. Fi

Delivery delays: 6 to 8 weeks.

 

”, and

 

 

Design Conte. Tel 11 9341. ~ 1000- Bruxelles
Galerie Ravenstein Galeri) $1 ~ 1000 - Brussel



In [14]:
def getSkewAngle(cvImage):
    newImage = cvImage.copy()
    gray = cv2.cvtColor(newImage, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Apply dilate to merge text into meaningful lines/paragraphs.
    # Use larger kernel on X axis to merge characters into single line, cancelling out any spaces.
    # But use smaller kernel on Y axis to separate between different blocks of text
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 5))
    dilate = cv2.dilate(thresh, kernel, iterations=5)

    # Find all contours
    contours, hierarchy = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)

    # Find largest contour and surround in min area box
    largestContour = contours[0]
    minAreaRect = cv2.minAreaRect(largestContour)

    # Determine the angle. Convert it to the value that was originally used to obtain skewed image
    angle = minAreaRect[-1]
    if angle < -45:
        angle = 90. + angle
    return -1 * angle + 90

In [15]:
# Rotate the image around its center
def rotateImage(cvImage, angle):
    newImage = cvImage.copy()
    (h, w) = newImage.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    newImage = cv2.warpAffine(newImage, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return newImage

# Deskew image
def deskew(cvImage):
    angle = getSkewAngle(cvImage)
    return rotateImage(cvImage, -1.0 * angle)

In [25]:
img = cv2.imread('QuixoteRot.png')
img2 = deskew(img)
plt.imshow(img2)
plt.show()

error: OpenCV(4.5.5) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0xa96199bf::Set<1,-1,-1>,struct cv::impl::A0xa96199bf::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


In [84]:
import pandas as pd
from pytesseract import Output
img = cv2.imread('ArrozYTartana/004.tif')
text = pytesseract.image_to_data(img, config=custom_config, output_type=Output.DATAFRAME).dropna()
#text['text'] = text['text'].lower()
list = text["text"].tolist()
try:
    while True:
        list.remove('')
except ValueError:
    pass
try:
    while True:
        list.remove(' ')
except ValueError:
    pass
list.pop(-1)
filterlist = []
skip = []
for i in range(len(list)):
    if i in skip:
        continue
    if list[i].find("-")!=-1:
        if i+1>=len(list):
            break
        else:
            filterlist.append(list[i].replace('-', '')+list[i+1])
            skip.append(i+1)
    else:
        filterlist.append(list[i])
print(filterlist)

['A', 'las', 'tres', 'de', 'la', 'tarde', 'entró', 'doña', 'Manuela', 'en', 'la', 'plaza', 'del', 'Mercado,', 'envuelto', 'el', 'airoso', 'busto', 'en', 'un', 'abrigo', 'cuyos', 'faldones', 'casi', 'llegaban', 'al', 'borde', 'de', 'la', 'falda,', 'cuidadosamente', 'enguantada,', 'con', 'el', 'limosnero', 'al', 'puño', 'y', 'velado', 'el', 'rostro', 'por', 'la', 'tenue', 'blonda', 'de', 'la', 'mantilla,', 'Tras', 'ella,', 'formando', 'una', 'pareja', 'silenciosa,', 'marchaban', 'el', 'cochero', 'y', 'la', 'criada:', 'un', 'mocetón', 'de', 'rostro', 'carrilludo', 'y', 'afeitado', 'que', 'respiraba', 'brutal', 'jocosidad,', 'luciendo', 'con', 'tanta', 'satisfacción', 'como', 'embarazo', 'los', 'pesados', 'borceguíes,', 'el', 'terno', 'azul', 'con', 'vivos', 'rojos', 'y', 'botones', 'dorados', 'y', 'la', 'gorra', 'de', 'hule', 'de', 'ancho', 'plato,', 'y', 'a', 'su', 'lado', 'una', 'muchacha', 'morena', 'y', 'guapota,', 'con', 'peinado', 'de', 'rodete', 'y', 'agujas', 'de', 'perlas,', 'com

In [85]:
import re
content = open("004.txt", "r", encoding='utf-8').read()
lecturatxt = re.split('\s|\n',content)
try:
    while True:
        lecturatxt.remove('')
except ValueError:
    pass
#lecturatxt.remove('')

205
205


In [86]:
precision_score(lecturatxt, filterlist, average='macro')

0.9694656488549618

In [105]:
content = open("004.txt", "r", encoding='utf-8').read()
print(type(content))

<class 'str'>


In [53]:
textos = pytesseract.image_to_string(cv2.imread('ArrozYTartana/005.tif'), config=custom_config)
print(textos)

transeúntes demasiado curiosos O a pasear ojea-
das hurañas de la señora al cochero o viceversa.

Cuando, doblando la esquina, entraron los tres
en la plaza del Mercado, doña Manuela se detuvo
como desorientada.

¡Gran Dios...! ¡cuánta gente! Valencia entera
estaba allí. Todos los años ocurría lo mistno en el
día de Nochebuena. Aquel mercado extraordina-
rio, que se prolongaba hasta bien entrada la no-
che, resultaba una festividad ruidosa, la explosión
de alegría y bullicio de un pueblo que entre mon-
tones de alimentos y aspirando el tufillo de las
mil cosas que satisfacen la voracidad humana, re-
gocijábase al pensar en los atracones del día si-
guiente. En aquella plaza larga, ligeramente ar-
queada y estrecha en sus extremos, como un in-
testino hinchado, amontonábanse las nubes de ali-
mentos que habían de desparramarse como nutri-
tiva lluvia sobre las mesas, satisfaciendo la gigan-
tesca gula de la Navidad, fiesta gastronómica, que
es como el estómago del año.

Doña Manuela per

In [106]:
img = cv2.imread('ArrozYTartana/005.tif')
textos = pytesseract.image_to_string(img, config=custom_config)
sin = re.split('\s|\n',textos)
print(sin)

['transeúntes', 'demasiado', 'curiosos', 'O', 'a', 'pasear', 'ojea-', 'das', 'hurañas', 'de', 'la', 'señora', 'al', 'cochero', 'o', 'viceversa.', '', 'Cuando,', 'doblando', 'la', 'esquina,', 'entraron', 'los', 'tres', 'en', 'la', 'plaza', 'del', 'Mercado,', 'doña', 'Manuela', 'se', 'detuvo', 'como', 'desorientada.', '', '¡Gran', 'Dios...!', '¡cuánta', 'gente!', 'Valencia', 'entera', 'estaba', 'allí.', 'Todos', 'los', 'años', 'ocurría', 'lo', 'mistno', 'en', 'el', 'día', 'de', 'Nochebuena.', 'Aquel', 'mercado', 'extraordina-', 'rio,', 'que', 'se', 'prolongaba', 'hasta', 'bien', 'entrada', 'la', 'no-', 'che,', 'resultaba', 'una', 'festividad', 'ruidosa,', 'la', 'explosión', 'de', 'alegría', 'y', 'bullicio', 'de', 'un', 'pueblo', 'que', 'entre', 'mon-', 'tones', 'de', 'alimentos', 'y', 'aspirando', 'el', 'tufillo', 'de', 'las', 'mil', 'cosas', 'que', 'satisfacen', 'la', 'voracidad', 'humana,', 're-', 'gocijábase', 'al', 'pensar', 'en', 'los', 'atracones', 'del', 'día', 'si-', 'guiente.', 

In [70]:
listaprueba = ['Fuck', 'you',  'Python']
print(listaprueba.remove('')=='ValueError: list.remove(x): x not in list')

ValueError: list.remove(x): x not in list

In [17]:
import os
directory = 'C:/Users/alons/Desktop/Universidad/Universidad21.22/TFG/Vision/ArrozYTartana'

for filename in os.listdir(directory):
    if filename.endswith(".tif"):
        print(filename.replace(".tif", ".txt"))
    listafilt = listafiltrada(filename.replace(".tif", ".txt"))
    txtleido = leertxt(filename)
    print(len(listafilt), len(txtleido))

TypeError: Unsupported image object